In [1]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, col, count, when
from pyspark.sql.functions import round as rnd

In [2]:
conf = SparkConf()
conf.set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.2.2')
conf.set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.2.2,com.microsoft.azure:spark-mssql-connector_2.12:1.2.0')
conf.set('spark.hadoop.fs.s3a.aws.credentials.provider', 'com.amazonaws.auth.InstanceProfileCredentialsProvider')
spark = SparkSession.builder.config(conf=conf).getOrCreate()

:: loading settings :: url = jar:file:/usr/local/lib/python3.7/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ec2-user/.ivy2/cache
The jars for the packages stored in: /home/ec2-user/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
com.microsoft.azure#spark-mssql-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-a09bab07-36b2-4923-9c62-9163a6709c77;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.2.2 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.563 in central
	found com.microsoft.azure#spark-mssql-connector_2.12;1.2.0 in central
	found com.microsoft.sqlserver#mssql-jdbc;8.4.1.jre8 in central
:: resolution report :: resolve 508ms :: artifacts dl 23ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.563 from central in [default]
	com.microsoft.azure#spark-mssql-connector_2.12;1.2.0 from central in [default]
	com.microsoft.sqlserver#mssql-jdbc;8.4.1.jre8 from central in [default]
	org.apache.hadoop#hadoop-aws;3.2.2 from central in [default]
	-------------------------

In [3]:
dfTweets = spark.read.csv("s3a://cf-murilo-pi-2022-01-sptech-bucket-tratados/tweets/tweets-tratados.csv", header=True,
                          inferSchema=True)
dfTweets.columns

22/06/08 01:58:09 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


['Tweets', 'len', 'ID', 'Date', 'Geo', 'Source', 'SA', 'Estado', 'Coordenadas']

In [4]:
dfIDD = spark.read.csv("s3a://cf-murilo-pi-2022-01-sptech-bucket-tratados/idd/idd-tratados.csv", header=True,
                       inferSchema=True)
dfIDD.columns

['IDD_Continuo',
 'IDD_Faixa',
 'Nota_Bruta_IDD',
 'Estado_Curso',
 'Instituicao_Ensino',
 'Concluintes_Inscritos',
 'Concluintes_Participantes',
 'Concluintes_Participantes_Nota_Enem',
 'Prop_Concluintes_Participantes_Nota_Enem_IDD',
 'Curso']

In [5]:
dfEnem = spark.read.csv("s3a://cf-murilo-pi-2022-01-sptech-bucket-tratados/enem/enem-tratados.csv", header=True,
                        inferSchema=True)
dfEnem.columns

['Treineiro',
 'Estado',
 'Presenca_CN',
 'Presenca_CH',
 'Presenca_LC',
 'Presenca_MT',
 'Nota_CN',
 'Nota_CH',
 'Nota_LC',
 'Nota_MT',
 'Status_Redacao',
 'Nota_Redacao']

In [6]:
dfEnemMediaNotaEstado = dfEnem.filter((dfEnem['Status_Redacao'] == 1) & (dfEnem['Presenca_CN'] == 1) & (dfEnem['Presenca_CH'] == 1) & (dfEnem['Presenca_LC'] == 1) & (dfEnem['Presenca_MT'] == 1)).groupby("Estado").agg(
    rnd(avg(col("Nota_CN")), 2).alias("Nota Ciências Naturais"),
    rnd(avg(col("Nota_CH")), 2).alias("Nota Ciências Humanas"),
    rnd(avg(col("Nota_LC")), 2).alias("Nota Linguagens e Códigos"),
    rnd(avg(col("Nota_MT")), 2).alias("Nota Matemática"),
    rnd(avg(col("Nota_Redacao")), 2).alias("Nota Redação"),
    count(when(col("Treineiro") == 1, True)).alias("Treineiros"),
    count(when(col("Treineiro") == 0, True)).alias("Não Treineiros"))

In [7]:
dfIddMediaEstado = dfIDD.groupby(col("Estado_Curso").alias("Estado")).agg(
    rnd(avg(col("IDD_Continuo")), 2).alias("IDD Continuo"),
    rnd(avg(col("IDD_Faixa")), 2).alias("IDD Faixa"),
    rnd(avg(col("Concluintes_Inscritos")), 2).alias("Concluintes Inscritos"),
    rnd(avg(col("Concluintes_Participantes")), 2).alias("Concluintes Participantes"),
    rnd(avg(col("Concluintes_Participantes_Nota_Enem")), 2).alias("Concluintes Participantes Nota Enem"),
    rnd(avg(col("Prop_Concluintes_Participantes_Nota_Enem_IDD")), 2).alias(
        "Prop Concluintes Participantes Nota Enem IDD"))

In [8]:
dfTweetsSAEstado = dfTweets.groupby("Estado").agg(
    count(when(col("SA") > 0, True)).alias("Tweets Positivos"),
    count(when(col("SA") < 0, True)).alias("Tweets Negativos"),
    count(when(col("SA") == 0, True)).alias("Tweets Neutros"))

In [9]:
dfInnerJoin = dfEnemMediaNotaEstado.join(dfIddMediaEstado, on="Estado", how="inner").join(
    dfTweetsSAEstado, on="Estado", how="outer")

In [10]:
dfInnerJoin.count()


29

In [11]:
dfInnerJoin.coalesce(1).write.option("header", True).csv("s3a://cf-murilo-pi-2022-01-sptech-bucket-cliente/dados", emptyValue='')

22/06/08 01:58:53 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.
22/06/08 01:58:58 WARN AbstractS3ACommitterFactory: Using standard FileOutputCommitter to commit work. This is slow and potentially unsafe.


In [13]:
dfInnerJoin.write \
 .format('com.microsoft.sqlserver.jdbc.spark') \
 .mode('overwrite') \
 .option('driver', 'com.microsoft.sqlserver.jdbc.SQLServerDriver') \
 .option('url', 'jdbc:sqlserver://bancopi-murilo.database.windows.net;databaseName=banco-pi;') \
 .option('dbtable', 'dbo.Dados') \
 .option('user', 'MuriloAC') \
 .option('password', 'Urubu@100') \
 .save()